In [ ]:
!pip install -q efficientnet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random
import cv2
from PIL import Image

import tensorflow as tf
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn

from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
cfg = dict(
           batch_size=32,
           img_size=350,
    
           lr_start=0.000005,
           lr_max=0.00000125,
           lr_min=0.000001,
           lr_rampup=5,
           lr_sustain=0,
           lr_decay=0.8,
           epochs=12,
    
           transform_prob=1.0,
           rot=180.0,
           shr=2.0,
           hzoom=8.0,
           wzoom=8.0,
           hshift=8.0,
           wshift=8.0,
    
           optimizer='adam',
           label_smooth_fac=0.05,
           tta_steps=20
            
        )

In [ ]:
BASE_PATH = '/kaggle/input/human-dataset/data/'
BASE_PATH = '/kaggle/input/human-dataset-latest/human-dataset/data/'

In [ ]:
lis = os.listdir(BASE_PATH+'train/train/0/')
lis2 = os.listdir(BASE_PATH+'train/train/1/')

In [ ]:
rows = 3
cols = 5
fig,axis = plt.subplots(rows,cols,figsize=(20,15))

for row in range(rows):
    for col in range(cols):
        axis[row][col].imshow(Image.open(BASE_PATH+"train/train/0/"+lis[random.randint(0,len(lis))]))

In [ ]:
rows = 3
cols = 5
fig,axis = plt.subplots(rows,cols,figsize=(20,15))

for row in range(rows):
    for col in range(cols):
        axis[row][col].imshow(Image.open(BASE_PATH+"train/train/1/"+lis2[random.randint(0,len(lis2))]))

In [ ]:
input_l = tf.keras.Input(shape=(cfg['img_size'], cfg['img_size'], 3),
                             name='img_input')



x = efn.EfficientNetB3(include_top=False,
                       weights='imagenet',
                       input_shape=(cfg['img_size'], cfg['img_size'], 3),
                       pooling='avg')(input_l)

output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(input_l,output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.Precision(name='Precision'),tf.keras.metrics.Recall(name='recall'),'accuracy'])

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.3,
        #rotation_range=10,
        #width_shift_range=[-100,100],
        #height_shift_range=0.4,
        #brightness_range=[0.2,0.5],
        #horizontal_flip=True,
        validation_split=0.3)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        BASE_PATH+'train/train',
        target_size=(350, 350),
        batch_size=16,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        BASE_PATH+'train/train',
        target_size=(350, 350),
        batch_size=16,
        subset='validation',
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        BASE_PATH+'val/human detection dataset/',
        target_size=(350, 350),
        batch_size=16,
        class_mode='binary')

In [ ]:
!mkdir weights

In [ ]:
def getLearnRateCallback(cfg):
    
    lr_start = cfg['lr_start']
    lr_max = cfg['lr_max'] * strategy.num_replicas_in_sync * cfg['batch_size']
    lr_min = cfg['lr_min']
    lr_rampup = cfg['lr_rampup']
    lr_sustain = cfg['lr_sustain']
    lr_decay = cfg['lr_decay']

    def lrfn(epoch):
        if epoch < lr_rampup:
            lr = (lr_max - lr_start) / lr_rampup * epoch + lr_start
        elif epoch < lr_rampup + lr_sustain:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_rampup -
                                                lr_sustain) + lr_min
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

lr_scheduler = getLearnRateCallback(cfg)

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8)

checkpoint_path = "weights/"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)

In [ ]:
history = model.fit(train_generator,
        epochs=50,
        validation_data=validation_generator,
        callbacks=[lr_scheduler,earlystopping,cp_callback])

In [ ]:
model.save("model_new_with_test.h5")

In [ ]:
#model = tf.keras.models.load_model('/kaggle/input/human-detection-trained-model/model-70-30-split-good valid-accuracy-with-worst-test accuracy.h5')

In [ ]:
actual = []
predicted = []

for index in tqdm(range(len(validation_generator))):
    batch = validation_generator[index]
    for image in batch[0]:
        res = 1*(model.predict(np.array([image])) >= 0.49)
        predicted.append(res)
    actual.extend(batch[1])
    
predicted = [x[0][0] for x in predicted]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(actual,predicted))

In [ ]:
actual = []
predicted = []

for index in tqdm(range(len(test_generator))):
    batch = test_generator[index]
    for image in batch[0]:
        res = 1*(model.predict(np.array([image])) >= 0.49)
        predicted.append(res)
    actual.extend(batch[1])
    
predicted = [x[0][0] for x in predicted]

print(classification_report(actual,predicted))

In [ ]:
!cp -r '/kaggle/input/yolov3' '/kaggle/working/'

In [ ]:
!cp '/kaggle/input/yolo-v3-weights/yolo.h5' '/kaggle/working/yolov3/YOLOv3/data/'

In [ ]:
cd yolov3/YOLOv3

In [ ]:
import os
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
from model.yolo_model import YOLO

In [ ]:
def process_image(img):

    image = cv2.resize(img, (416, 416),interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [ ]:
def draw(image, boxes, scores, classes, all_classes):

    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[0], score),(top, left - 6),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0, 0, 255), 1,cv2.LINE_AA)

        #print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
     #   print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [ ]:
def detect_image(image, yolo, all_classes):
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()
    
    res = 0
    
    if classes is not None:
        if 0 in classes:
            res = 1
            #print('Intruder Alert')

    #print('time: {0:.2f}s'.format(end - start))

    #if boxes is not None:
    #    draw(image, boxes, scores, classes, all_classes)
    return res


In [ ]:
yolo = YOLO(0.6, 0.5)
all_classes = ['person']

In [ ]:
actual = []
predicted = []

for index in tqdm(range(len(validation_generator))):
    batch = validation_generator[index]
    
    for image in batch[0]:
        image = ((image - image.min()) * (1/(image.max() - image.min()) * 255)).astype('uint8')
        res = detect_image(image, yolo, all_classes)
        predicted.append(res)
    
    actual.extend(batch[1])

In [ ]:
print(classification_report(actual,predicted))

In [ ]:
actual = []
predicted = []

for index in tqdm(range(len(test_generator))):
    batch = test_generator[index]
    
    for image in batch[0]:
        image = ((image - image.min()) * (1/(image.max() - image.min()) * 255)).astype('uint8')
        res = detect_image(image, yolo, all_classes)
        predicted.append(res)
    
    actual.extend(batch[1])

In [ ]:
print(classification_report(actual,predicted))